## Download from Cepik api for motoliczby.pl

Dokumnetacja APi Cepik: https://api.cepik.gov.pl/doc
Filtrowanie: filter[<nazwa atrybutu>]=<szukana wartość>
    
    

In [1]:
import pandas as pd
import numpy as np
import json
import time
import requests
import os
import math
from datetime import datetime

In [2]:
# names of Wojewodztwa
wojew_names = {
"02" : "DOLNOŚLĄSKIE",
"04" : "KUJAWSKO-POMORSKIE",
"06" : "LUBELSKIE",
"08" : "LUBUSKIE",
"10" : "ŁÓDZKIE",
"12" : "MAŁOPOLSKIE",
"14" : "MAZOWIECKIE",
"16" : "OPOLSKIE",
"18" : "PODKARPACKIE",
"20" : "PODLASKIE",
"22" : "POMORSKIE",
"24" : "ŚLĄSKIE",
"26" : "ŚWIĘTOKRZYSKIE",
"28" : "WARMIŃSKO-MAZURSKIE",
"30" : "WIELKOPOLSKIE",
"32" : "ZACHODNIOPOMORSKIE",
"XX" : "NIEOKREŚLONE"
}




In [3]:
#wojew = ["02","04","06","08","10","12","14","16","18","20","22","24","26","28","30","32","XX"]
#years = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2023"]

# Settings of years and 

wojew = ["02","04","06","08","10","12","14","16","18","20","22","24","26","28","30","32","XX"]
years = ["2023"]



In [4]:
# Settings
main_data_path = "C:/Users/tcwik/Documents/Python_Scripts/cepik/test/" # please define here a upload path.
current_dic = "motocykle" # Define a folder
now_file = "generated_" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
upload_path = main_data_path + current_dic + "_" + now_file

base_name_xls = "Cepik_" # base name of the xls/csv file
path_to_merge = upload_path + current_dic + "all/" # name of concated date file
all_merged_data_file_name ="Cepik_all_data_in_one_file.csv" # name of the file with all data


In [5]:
# url and filters

based_url = "https://api.cepik.gov.pl/pojazdy?&limit=500" 
filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false'
#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false&filter[rodzaj-pojazdu]=Motocykl&filter[marka]=bmw&filter[model]=K1'

#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false&filter[rodzaj-pojazdu]=Motocykl'
# old filters

#period = "&data-od=20100101&data-do=20111231"
# Filtry
# spacebar &nbsp;
#filtr = '&filter[pochodzenie-pojazdu]=NOWY+&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true'
#filtr = '&filter[pochodzenie-pojazdu]=NOWY&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true'
#filtr = '&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true'
#filtr = '&filter[pochodzenie-pojazdu]=NOWY+ZAKUPIONY+W+KRAJU&filter[rodzaj-pojazdu]=SAMOCHÓD+OSOBOWY&pokaz-wszystkie-pola=true&sort=marka&sort=model&sort=typ&sort=wariant'
#filtr = '&filter[pochodzenie-pojazdu]=NOWY+ZAKUPIONY+W+KRAJU&filter[pochodzenie-pojazdu]=NOWY+IMPORT+INDYW&filter[pochodzenie-pojazdu]=NOWY+ZAKUPIONY+W+KRAJU&pokaz-wszystkie-pola=true&sort=id'
# --------------
# main filtr
#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false'
#elektryczne
#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA'
# --------------
#filtr ="&filter[rodzaj-pojazdu]=SAMOCHÓD+OSOBOWY"

#filtr = '&filter[marka]=TOYOTA&filter[pochodzenie-pojazdu]=NOWY+ZAKUPIONY+W+KRAJU&filter[pochodzenie-pojazdu]=NOWY+IMPORT+INDYW&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true'
# ------- path ---------
#upload_path = "J:/Dokumenty/Tomek/Data_science/Cepik/nowe_samochody/"
#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false&rodzaj-pojazdu=Motocykl&marka=bmw&model=K1'
#"UŻYW. IMPORT INDYW","UŻYW. ZAKUPIONY W KRAJU","PONOWNA REJESTRACJA"



In [6]:
# data frames declaration
all_cars_in_years = pd.DataFrame()
cars_in_years_and_woj = pd.DataFrame()



In [7]:
try:
    del cars
    del df
except Exception:
    print("no df to del")

no df to del


In [8]:
# main Cell

# years loop
for y in years:
#for w in wojew:

    print ("-"*40)
    print ("Scope of data capture (years): start:", years[0] ,"end:", years[-1] )
    print("Year:", y)
    
    # Wojewodztwo loop
    for w in wojew:
    
                
        # variables
        wojewodz = "&wojewodztwo=" + w
        period = "&data-od=" + y + "0101&data-do=" + y + "1231"
        path_to_save = upload_path + "/" + y + "/" + w + "/" 
        path_to_save_all = upload_path + "/" + y + "/" 
        
        rec_no = 0
        step = 1
        finished = False 
        number_of_waits = 0
        page = "&page=1" 
        
    
        #prints 
        print ("-"*20)
        print("Woj:", w," ",wojew_names[w], "/",y)
        
               
        # define first page
        url = based_url + period +  filtr + wojewodz + page
        
        # scrapping loop within the Woj
        while not finished:
            
            print ("We dig in:", url)

            #------------------------------------    
            data_temp = requests.get(url) 
            #------------------------------------
            
            # check the respon code if no errors
            if data_temp.status_code != 200:
                number_of_waits = number_of_waits + 1
                print ("digging status NOK: ", data_temp.status_code, "wait for 5 sek.", number_of_waits , "times")
                tc_retry = True
                time.sleep(5) # czekam 5 sek
                
            else:
                tc_retry = False
                print ("digging OK -", "data statatus code:",data_temp.status_code)
                
                data = data_temp.json()
                df = pd.json_normalize(data, record_path =['data'])

                # how many times we need to iterate data to download
                tc_links = pd.json_normalize(data)
                                  
                # numbers of records
                rec_no = tc_links["meta.count"][0]
                records_tc =  rec_no
                rec_no = math.ceil(rec_no/500)
                                
                # Stop the loop if there is no records to downlaod
                if records_tc == 0:
                    print ("----NO RECORDS DO DOWNLOAD","page/step: ",step,"/", rec_no," Records in woj: ",w, "records to download:",records_tc ," pages remaining: ", rec_no - step)
                    break;
                                
                # Make a directory if thera are records to save
                if records_tc != 0 and step == 1:
                    try: 
                        os.makedirs (path_to_save)
                        print (path_to_save, " successfully created")
                    except Exception:
                        print("dic: ", path_to_save, "aready exist" )

                
                # Define next url to capture
                try: 
                    next_url = tc_links["links.next"][0]
                except Exception:
                    next_url = "No next url - it was last one"
                    
                # define next url to download    
                last_url = tc_links["links.last"][0]
              
                
                                
                # ----------- append data into dataframes of woj ----------------
                cars_in_years_and_woj = pd.concat([cars_in_years_and_woj, df])
                
                # Printouts
                #print ("url :", url )
                #print ("next:",next_url )
                #print ("last:",last_url )
                print (">>page/step: ",step,"/", rec_no," Records in woj: ",w, "records to download:",records_tc , "total rec:", len(cars_in_years_and_woj)," Unique:", cars_in_years_and_woj['id'].nunique(), " pages remaining: ", rec_no - step)
                #print ("total rec:", len(cars_in_years_and_woj)," Unique:", cars_in_years_and_woj['id'].nunique())
                

                
                # save each set of data from particular step steps into the csv
                now_file = "generated_" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
                name_xls = path_to_save +        base_name_xls + y + "_woj_" +w + "_step_" + str(step) + "-" + now_file + ".csv"
                name_xls_total = path_to_save_all + base_name_xls + y + "_woj_" +w + "_total_" + "-" + now_file + ".csv"
                
                df.to_csv(path_or_buf = name_xls,
                            index = False,
                            sep = ";",
                            encoding = 'utf-8')
                #print ("total rec:", len(df)," Unique:", df['id'].nunique())



                if last_url == url or next_url == "No next url - it was last one":
                    
                    
                    finished = True

                    # Save df                 
                    cars_in_years_and_woj.to_csv(
                                        path_or_buf= name_xls_total,
                                        index = False,
                                        sep = ";",
                                        encoding = 'utf-8')
                    
                    # printouts end of the wojewodztwo
                    print ("*"*60)
                    print ("End of Wojewodztwo. Wojewodztowo ",w,"/",y," is downloaded")
                    print ("File saved: total rec:", len(cars_in_years_and_woj)," Unique:", cars_in_years_and_woj['id'].nunique())
                    print ("*"*60)

                    # ----------- append data into dataframes of year ----------------
                    #all_cars_in_years = all_cars_in_years.append(cars_in_years_and_woj)
                    all_cars_in_years = pd.concat([all_cars_in_years , cars_in_years_and_woj])
                    


                else:    
                    step = step + 1
                    url = next_url



                print ("-"*5)

    
  
    
    
    # Save all data within a year   
    if len(cars_in_years_and_woj) > 0:
        
        # Verification of duplicates
        cars_in_years_and_woj_records = len(cars_in_years_and_woj)
        cars_in_years_and_woj_unique = cars_in_years_and_woj['id'].nunique()
        
        cars_in_years_and_woj.to_csv(
            path_or_buf= path_to_save_all + base_name_xls + y + "_all_woj_total_" + "-" + now_file + ".csv",
            index = False,
            sep = ";",
            encoding = 'utf-8')

        # Printouts end of the year
        print("-------------------------------------")
        print("------ Year "  + y + " Sucessfuly downloaded to",path_to_save_all)
        print("Total records: ", cars_in_years_and_woj_records)
        print("Total unique : ", cars_in_years_and_woj_unique)
        if cars_in_years_and_woj_records == cars_in_years_and_woj_unique:
            print ("OK no duplicates found")
        else:
            print ("!!!! Possible duplicates, please check it!")
        print("-------------------------------------")
    else:
        print("No records found in: ",y," year")

    # end of Year loop

----------------------------------------
Scope of data capture (years): start: 2023 end: 2023
Year: 2023
--------------------
Woj: 02   DOLNOŚLĄSKIE / 2023
We dig in: https://api.cepik.gov.pl/pojazdy?&limit=500&data-od=20230101&data-do=20231231&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false&wojewodztwo=02&page=1
digging OK - data statatus code: 200
C:/Users/tcwik/Documents/Python_Scripts/cepik/test/motocykle_generated_2023_05_22_13_53_23/2023/02/  successfully created
>>page/step:  1 / 83  Records in woj:  02 records to download: 41177 total rec: 500  Unique: 500  pages remaining:  82
-----
We dig in: https://api.cepik.gov.pl/pojazdy?wojewodztwo=02&data-od=20230101&data-do=20231231&limit=500&page=2&typ-daty=1&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true&sort=-id
digging OK - data statatus code: 200
>>page/step:  2 / 83  Records in woj:  02 records to download: 41177 total rec: 1000  Unique: 1000  pages remaining:  81
-----
We dig in: https://api.cepik.gov.pl/poja

KeyboardInterrupt: 

In [ ]:
print ("koniec")